### 1. Question: What factors most influence student performance?
Objective: Determine which factors (e.g., attendance, specific subjects, types of evaluation) most significantly correlate with student performance.

### 2. Question: Can we predict student dropout or failure based on early performance indicators?
Objective: Build a predictive model to identify students at risk of dropping out or failing.
- Tables to Use:
    - TbSituacaoAlunoDisciplina
    - TbMetaFaseNotaAluno
    - TbMeta
    - TbDiarioFrequencia
- Approach:
    - Join tables to gather historical performance and attendance data.
    - Use classification algorithms (e.g., logistic regression, random forest) to predict dropout or failure risk.


In [15]:
import pandas as pd

# Load one of the CSV files
tb_aluno = pd.read_csv('csv_output/Tabelas/TbAluno/Originais anonimizados/TbAluno.csv')

# Example: Joining TbAluno with TbDiarioFrequencia
tb_diario_frequencia = pd.read_csv('csv_output/Tabelas/TbDiario/Originais anonimizados/TbDiarioFrequencia.csv')

# Merge on IdAluno
merged_data = pd.merge(tb_aluno, tb_diario_frequencia, on='IdAluno')

# Analyze attendance by student
attendance_summary = merged_data.groupby('IdAluno')['StPresencaFalta'].value_counts()

attendance_summary


IdAluno  StPresencaFalta
3        P                  158
         F                   24
4        F                   70
         P                   48
5        P                   16
                           ... 
2237     P                   12
2238     P                   11
         J                    1
2241     P                    8
         F                    4
Name: count, Length: 5163, dtype: int64

In [21]:
  # Drop columns with high percentage of missing values
missing_series = merged_data.isnull().mean()
columns_to_drop = missing_series[missing_series > 0.5].index
merged_data = merged_data.drop(columns=columns_to_drop)
merged_data_cleaned = merged_data.drop(columns=[ 'NomeAluno'])

 # Select only numeric columns for variance and correlation checks
numeric_df = merged_data_cleaned.select_dtypes(include=['number'])

# Drop columns with low variance
variance_series = numeric_df.var()
columns_to_drop = variance_series[variance_series < 0.01].index
df = merged_data_cleaned.drop(columns=columns_to_drop)
df

# Drop columns with only one unique value
unique_value_counts = df.nunique()
columns_to_drop = unique_value_counts[unique_value_counts == 1].index
df = df.drop(columns=columns_to_drop)
print(f"Dropped columns with only one unique value: {list(columns_to_drop)}")

df_cleaned = df.drop(columns=['Sexo', 'IdMae', 'IdResponsavel', 'IdTipoResponsavel', 'CorRaca'])
df_cleaned

Dropped columns with only one unique value: ['StDeficienciaCegueira', 'StDeficienciaBaixaVisao', 'StDeficienciaSurdez', 'StDeficienciaAuditiva', 'StDeficienciaFisica', 'StDeficienciaSurdoCegueira', 'StDeficienciaMultipla', 'StDeficienciaMental', 'StFalecido', 'StDeficienciaAutismoInfantil', 'StDeficienciaSindromeAsperger', 'StDeficienciaSindromeRett', 'StDeficienciaTrastornoDesintegrativo', 'StDeficienciaAltasHabilidades', 'StAppComunicacao_Sincronizado', 'StRecursoAuxilioLedor', 'StRecursoAuxilioTranscricao', 'StRecursoGuiaInterprete', 'StRecursoTradutorInterpreteDeLibras', 'StRecursoLeituraLabial', 'StRecursoMaterialDidaticoProvaBraille', 'StRecursoProvaAmpliada', 'StRecursoProvaSuperampliada', 'StRecursoCdComAudio', 'StRecursoLinguaPortuguesaSegundaLingua', 'StRecursoProvaEmVideo', 'StPermiteUsoImagem', 'StEstrangeiro']


,IdAluno,EstadoCivil,DataNascimento,IdDiarioFrequencia,IdDiarioAula,StPresencaFalta
0,3,5.0,2005-03-29 03:00:00,440,36,P
1,3,5.0,2005-03-29 03:00:00,441,37,P
2,3,5.0,2005-03-29 03:00:00,442,38,P
3,3,5.0,2005-03-29 03:00:00,443,39,P
4,3,5.0,2005-03-29 03:00:00,1794,299,F
...,...,...,...,...,...,...
313104,2241,5.0,2016-04-25 00:00:00,312342,24894,P
313105,2241,5.0,2016-04-25 00:00:00,313278,24937,F
313106,2241,5.0,2016-04-25 00:00:00,314361,25045,P
313107,2241,5.0,2016-04-25 00:00:00,316487,25204,P


In [29]:
# Convert the birthdate column to datetime format
df_cleaned['DataNascimento'] = pd.to_datetime(df_cleaned['DataNascimento'], errors='coerce')


# Calculate age by subtracting the birthdate from the current date
today = pd.to_datetime('today')
df_cleaned


df_cleaned_idade = df_cleaned.drop(columns=['DataNascimento'])
df_cleaned_idade

,IdAluno,EstadoCivil,IdDiarioFrequencia,IdDiarioAula,StPresencaFalta,Idade
0,3,5.0,440,36,P,19.0
1,3,5.0,441,37,P,19.0
2,3,5.0,442,38,P,19.0
3,3,5.0,443,39,P,19.0
4,3,5.0,1794,299,F,19.0
...,...,...,...,...,...,...
313104,2241,5.0,312342,24894,P,8.0
313105,2241,5.0,313278,24937,F,8.0
313106,2241,5.0,314361,25045,P,8.0
313107,2241,5.0,316487,25204,P,8.0


In [33]:
df_cleaned_idade.groupby('IdAluno')['StPresencaFalta'].value_counts()

IdAluno  StPresencaFalta
3        P                  158
         F                   24
4        F                   70
         P                   48
5        P                   16
                           ... 
2237     P                   12
2238     P                   11
         J                    1
2241     P                    8
         F                    4
Name: count, Length: 5163, dtype: int64

In [38]:
# Assuming your DataFrame is named df
df_cleaned_idade['Presenca'] = df_cleaned_idade['StPresencaFalta'].apply(lambda x: 1 if x == 'P' else 0)
df_cleaned_idade['Falta'] = df_cleaned_idade['StPresencaFalta'].apply(lambda x: 1 if x == 'F' or x == 'J' else 0)

# Group by IdAluno and aggregate counts
presence_absence_counts = df_cleaned_idade.groupby(['IdAluno']).agg(
    Total_Presenca=('Presenca', 'sum'),
    Total_Falta=('Falta', 'sum')
).reset_index()

# Display the result
presence_absence_counts


,IdAluno,Total_Presenca,Total_Falta
0,3,158,24
1,4,48,70
2,5,16,11
3,6,0,27
4,7,1,26
...,...,...,...
2127,2235,15,0
2128,2236,14,0
2129,2237,12,0
2130,2238,11,1
